# Stable Diffusion 1.5 using torch.compile with OpenVINO backend

In [ ]:
import sys

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import openvino.frontend.pytorch.torchdynamo.backend
import time
import os

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Using full precision (FP32) model in CPU with `StableDiffusionPipeline`
[back to top ⬆️](#Table-of-contents:)

In [ ]:
generator = torch.Generator("cpu").manual_seed(1024)
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe.unet = torch.compile(pipe.unet, backend="openvino")
pipe.vae.decode = torch.compile(pipe.vae.decode, backend="openvino")    

In [ ]:
prompt = "A cute squirrel in the forest, portrait, 8k"
start_time = time.time()
image = pipe(prompt, num_inference_steps=50, generator=generator).images[0]
end_time = time.time()
print("Time taken: ", end_time - start_time)   
image